In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#importing shutil for file copying
import shutil

Making the required folder from the existing code. Don't be dumb reuse code, sometimes.

In [ ]:
# unziping the train zip and test zip file after copying them to the working folder 
# remember in kaggle input folder have been given only read only acess by the admin
# hence we copy in the working folder and then complete our job
!rm -rf ./train ./cats_and_dogs_small ./train.zip ./cats_and_dogs_small_2.h5 ./cats_and_dogs_small_1.h5 ./cats_and_dogs_small_3.h5
# making the copy of the test and train dir from the input dir since it is read-only
!ls -lrt
# !cp /kaggle/input/dogs-vs-cats/test1.zip /kaggle/working
# !unzip /kaggle/working/test1.zip 
!cp /kaggle/input/dogs-vs-cats/train.zip /kaggle/working
!unzip /kaggle/working/train.zip
!cd /kaggle/working
# use this block wisely

# checking the contents of my working directory
# use this wisely
for dirname, _, filenames in os.walk('/kaggle/working'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
!cd /kaggle/working
!ls -lrt

# first lets set the base paths and make the necessary directories for working out our problem
original_dataset_dir = '/kaggle/working'
base_dir = '/kaggle/working/cats_and_dogs_small'
os.mkdir(base_dir)
train_dir = os.path.join(base_dir,'train')
os.mkdir(train_dir)
validation_dir = os.path.join(base_dir, 'validation')
os.mkdir(validation_dir)
test_dir = os.path.join(base_dir, 'test')
os.mkdir(test_dir)

# now lets make the cats and dogs directories in them for housing the cats and the dogs images
train_cats_dir = os.path.join(train_dir, 'cats')
os.mkdir(train_cats_dir)
train_dogs_dir = os.path.join(train_dir, 'dogs')
os.mkdir(train_dogs_dir)
validation_cats_dir = os.path.join(validation_dir, 'cats')
os.mkdir(validation_cats_dir)
validation_dogs_dir = os.path.join(validation_dir, 'dogs')
os.mkdir(validation_dogs_dir)
test_cats_dir = os.path.join(test_dir, 'cats')
os.mkdir(test_cats_dir)
test_dogs_dir = os.path.join(test_dir, 'dogs')
os.mkdir(test_dogs_dir)

# Now we will be copying the actual data from the base directory to the working directories
fnames = ['cat.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(os.path.join(original_dataset_dir,'train'), fname)
    dst = os.path.join(train_cats_dir, fname)
    shutil.copyfile(src, dst)
fnames = ['cat.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(os.path.join(original_dataset_dir,'train'), fname)
    dst = os.path.join(validation_cats_dir, fname)
    shutil.copyfile(src, dst)
fnames = ['cat.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(os.path.join(original_dataset_dir,'train'), fname)
    dst = os.path.join(test_cats_dir, fname)
    shutil.copyfile(src, dst)
fnames = ['dog.{}.jpg'.format(i) for i in range(1000)]
for fname in fnames:
    src = os.path.join(os.path.join(original_dataset_dir,'train'), fname)
    dst = os.path.join(train_dogs_dir, fname)
    shutil.copyfile(src, dst)
fnames = ['dog.{}.jpg'.format(i) for i in range(1000, 1500)]
for fname in fnames:
    src = os.path.join(os.path.join(original_dataset_dir,'train'), fname)
    dst = os.path.join(validation_dogs_dir, fname)
    shutil.copyfile(src, dst)
fnames = ['dog.{}.jpg'.format(i) for i in range(1500, 2000)]
for fname in fnames:
    src = os.path.join(os.path.join(original_dataset_dir,'train'), fname)
    dst = os.path.join(test_dogs_dir, fname)
    shutil.copyfile(src, dst)
    
    
# Now for ease of use I will do something
# since the train folder contains too many images which we don't need I will delete it.
!rm -rf ./train

!rm -rf ./train.zip

Checking the folders made and their content.

In [ ]:
# checking the that data have been properly copied or not
print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total validation cat images:', len(os.listdir(validation_cats_dir)))
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))
print('total test cat images:', len(os.listdir(test_cats_dir)))
print('total test dog images:', len(os.listdir(test_dogs_dir)))

# Visualization of ConvNet Learning

It’s often said that deep-learning models are “black boxes”: learning representations that are difficult to extract and present in a human-readable form. Although this is partially true for certain types of deep-learning models, it’s definitely not true for convnets. The representations learned by convnets are highly amenable to visualization, in large part because they’re representations of visual concepts. Since 2013, a wide array of techniques have been developed for visualizing and interpreting these representations. We won’t survey all of them, but we’ll cover three of the most accessible and useful ones:

* Visualizing intermediate convnet outputs (intermediate activations)—Useful for understanding how successive convnet layers transform their input, and for getting a first idea of the meaning of individual convnet filters.

* Visualizing convnets filters—Useful for understanding precisely what visual pattern or concept each filter in a convnet is receptive to.

* Visualizing heatmaps of class activation in an image—Useful for understanding which parts of an image were identified as belonging to a given class, thus allowing you to localize objects in images.


**Visualizing intermediate activations**
Visualizing intermediate activations consists of displaying the feature maps that are output by various convolution and pooling layers in a network, given a certain input (the output of a layer is often called its activation, the output of the activation function). This gives a view into how an input is decomposed into the different filters learned by the network. You want to visualize feature maps with three dimensions: width, height, and depth (channels). Each channel encodes relatively independent features, so the proper way to visualize these feature maps is by independently plotting the contents of every channel as a 2D image. 

First let's start by loading the saved model from the previous example:

In [ ]:
from keras.models import load_model # usiing load_model to the load the model from a different notebook
model = load_model('/kaggle/input/cats-vs-dogs-models/cats_and_dogs_small_2 (2).h5')
model.summary()  # checking the model summary

Now , we will get a picture of a cat, not part of the images the network was trained on.

In [ ]:
img_path = '/kaggle/working/cats_and_dogs_small/train/cats/cat.999.jpg' # seting the image path

from keras.preprocessing import image # image from keras to preprocess the image

img = image.load_img(img_path, target_size=(150, 150)) # loading the image and resizing it into the target size of 150 x 150 pixels
img_tensor = image.img_to_array(img) # converting the image into a image array
img_tensor = np.expand_dims(img_tensor, axis=0) # changing the shape of the array to give the tensor its required $D shape
img_tensor /= 255.  # converting the value of the tensor as float values ranged between 0 and 1
# Its shape is (1, 150, 150, 3)
print(img_tensor.shape) # checking the shape of the tensor

In [ ]:
 # importing mathplolib and checking the picture we are gonna use
import matplotlib.pyplot as plt
plt.imshow(img_tensor[0])
plt.show()

In order to extract the feature maps you want to look at, you’ll create a Keras model that takes batches of images as input, and outputs the activations of all convolution and pooling layers. To do this, you’ll use the Keras class Model. A model is instantiated using two arguments: an input tensor (or list of input tensors) and an output tensor (or list of output tensors). The resulting class is a Keras model, just like the Sequential models you’re familiar with, mapping the specified inputs to the specified outputs. What sets the Model class apart is that it allows for models with multiple outputs, unlike Sequential. 

In [ ]:
# import models from keras 
from keras import models
layer_outputs = [layer.output for layer in model.layers[:8]] # storing the output of the 8 layers in a list
activation_model = models.Model(inputs=model.input, outputs=layer_outputs) # instatiating the input and output layers 

In [ ]:
# making prediction on the image
activations = activation_model.predict(img_tensor)
# storing all the activations in the first layer 
first_layer_activation = activations[0]
print(first_layer_activation.shape)

In [ ]:
# checking out some of the activation by the filers
plt.matshow(first_layer_activation[0, :, :, 4], cmap='viridis')
plt.matshow(first_layer_activation[0, :, :, 7], cmap='viridis')

You’ll extract and plot every channel in each of the eight activation maps, and you’ll stack the results in one big image tensor, with channels stacked side by side.

In [ ]:
# now we will plot all the activations of all the layers in the base of our pretrained network

layer_names = []

for layer in model.layers[:8]:
    layer_names.append(layer.name)  # appending layer names in a list
    
images_per_row = 16 # setting the number of activations we will align in each network

# instantiating a loop to create the plot that will show the activations


for layer_name, layer_activation in zip(layer_names, activations):
    n_features = layer_activation.shape[-1] # storing the number of features
    
    size = layer_activation.shape[1] # storing the activation size
    
    n_cols = n_features // images_per_row # calculating the number of colums based on number of features
    
    display_grid = np.zeros((size * n_cols, images_per_row * size)) # making a Tensor of zeros
    
    # iterating through the each elemet in the tensor and setting their values
    for col in range(n_cols):
        for row in range(images_per_row):
            # storing the layer activation and applying some mathematical operation on them
            channel_image = layer_activation[0,
                                             :,
                                             :,
                                             col * images_per_row + row]
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size,
                         row * size : (row + 1) * size] = channel_image
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
    scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

There are a few things to note here:

* The first layer acts as a collection of various edge detectors. At that stage, the activations retain almost all of the information present in the initial picture. 

* As you go higher, the activations become increasingly abstract and less visually interpretable. They begin to encode higher-level concepts such as “cat ear” and “cat eye.” Higher presentations carry increasingly less information about the visual contents of the image, and increasingly more information related to the class of the image. 

* The sparsity of the activations increases with the depth of the layer: in the first layer, all filters are activated by the input image; but in the following layers, more and more filters are blank. This means the pattern encoded by the filter isn’t found in the input image. 

We have just evidenced an important universal characteristic of the representations learned by deep neural networks: the features extracted by a layer become increasingly abstract with the depth of the layer. The activations of higher layers carry less and less information about the specific input being seen, and more and more information about the target (in this case, the class of the image: cat or dog). A deep neural network effectively acts as an information distillation pipeline, with raw data going in (in this case, RGB pictures) and being repeatedly transformed so that irrelevant information is filtered out (for example, the specific visual appearance of the image), and useful information is magnified and refined (for example, the class of the image).
This is analogous to the way humans and animals perceive the world: after observing a scene for a few seconds, a human can  remember which abstract objects were present in it (bicycle, tree) but can’t remember the specific appearance of these objects. 

In fact, if you tried to draw a generic bicycle from memory, chances are you couldn’t get it even remotely right, even though you’ve seen thousands of bicycles in your lifetime.



**Visualizing convnet filters**

Another easy way to inspect the filters learned by convnets is to display the visual pattern that each filter is meant to respond to. This can be done with gradient ascent in input space: applying gradient descent to the value of the input image of a convnet so as to maximize the response of a specific filter, starting from a blank input image. The resulting input image will be one that the chosen filter is maximally responsive to.

The process is simple: you’ll build a loss function that maximizes the value of a given filter in a given convolution layer, and then you’ll use stochastic gradient descent to adjust the values of the input image so as to maximize this activation value. For instance, here’s a loss for the activation of filter 0 in the layer block3_conv1 of the VGG16 network, pretrained on ImageNet.

In [ ]:
# importing the VGG16 model from keras
from keras.applications import VGG16
from keras import backend as K # importing keras backed
import tensorflow as tf # importing tensorflow
tf.compat.v1.disable_eager_execution() # disabling eager execution for this problems sake

model = VGG16(weights='imagenet',
              include_top=False) # instantiating the model
layer_name = 'block3_conv1'
filter_index = 0
layer_output = model.get_layer(layer_name).output # getting the layer output 
loss = K.mean(layer_output[:, :, :, filter_index]) # applying mean on the output 

In [ ]:
model.summary() # checking the model summary to look into what we are dealing

To implement gradient descent, you’ll need the gradient of this loss with respect to the model’s input. To do this, you’ll use the gradients function packaged with the backend module of Keras.

A non-obvious trick to use to help the gradient-descent process go smoothly is to normalize the gradient tensor by dividing it by its L2 norm (the square root of the average of the square of the values in the tensor). This ensures that the magnitude of the updates done to the input image is always within the same range.



In [ ]:

grads = K.gradients(loss, model.input)[0] # calculating the gradient

grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5) # taking the mean square root value of the gradient

Now you need a way to compute the value of the loss tensor and the gradient tensor, given an input image. You can define a Keras backend function to do this: iterate is a function that takes a Numpy tensor (as a list of tensors of size 1) and returns a list of two Numpy tensors: the loss value and the gradient value.

In [ ]:
# computing the loss tensor and gradient tensor
iterate = K.function([model.input], [loss, grads]) 
loss_value, grads_value = iterate([np.zeros((1, 150, 150, 3))])

In [ ]:
# converting the input data
input_img_data = np.random.random((1, 150, 150, 3)) * 20 + 128.
step = 1.
for i in range(40):
    loss_value, grads_value = iterate([input_img_data])
    input_img_data += grads_value * step

The resulting image tensor is a floating-point tensor of shape (1, 150, 150, 3), with values that may not be integers within [0, 255]. Hence, you need to postprocess this tensor to turn it into a displayable image. You do so with the following straightforward utility function.

In [ ]:
# converting the tesnsor values such ythat it ranges between 0 and 1
def deprocess_image(x):
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1
    x += 0.5
    x = np.clip(x, 0, 1)
    x *= 255
    x = np.clip(x, 0, 255).astype('uint8')
    return x

Now you have all the pieces. Let’s put them together into a Python function that takes as input a layer name and a filter index, and returns a valid image tensor representing the pattern that maximizes the activation of the specified filter.

In [ ]:
def generate_pattern(layer_name, filter_index, size=150):
    # Build a loss function that maximizes the activation
    # of the nth filter of the layer considered.
    layer_output = model.get_layer(layer_name).output
    loss = K.mean(layer_output[:, :, :, filter_index])

    # Compute the gradient of the input picture wrt this loss
    grads = K.gradients(loss, model.input)[0]

    # Normalization trick: we normalize the gradient
    grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)

    # This function returns the loss and grads given the input picture
    iterate = K.function([model.input], [loss, grads])
    
    # We start from a gray image with some noise
    input_img_data = np.random.random((1, size, size, 3)) * 20 + 128.

    # Run gradient ascent for 40 steps
    step = 1.
    for i in range(40):
        loss_value, grads_value = iterate([input_img_data])
        input_img_data += grads_value * step
        
    img = input_img_data[0]
    
    return deprocess_image(img)

In [ ]:
# checking the use of function once 
# if it is working properly
plt.imshow(generate_pattern('block1_conv1', 0))


It seems that filter 0 in layer block3_conv1 is responsive to a polka-dot pattern. Now the fun part: you can start visualizing every filter in every layer. For simplicity, you’ll only look at the first 64 filters in each layer, and you’ll only look at the first layer of each convolution block (block1_conv1, block2_conv1, block3_conv1, block4_conv1, block5_conv1). You’ll arrange the outputs on an 8 × 8 grid of 64 × 64 filter patterns, with some black margins between each filter pattern.
    

In [ ]:
for layer_name in ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1']:
    size = 64
    margin = 5

    # This a empty (black) image where we will store our results.
    results = np.zeros((8 * size + 7 * margin, 8 * size + 7 * margin, 3), dtype = int)


    for i in range(8):  # iterate over the rows of our results grid
        for j in range(8):  # iterate over the columns of our results grid
            # Generate the pattern for filter `i + (j * 8)` in `layer_name`
            filter_img = generate_pattern(layer_name, i + (j * 8), size=size)

            # Put the result in the square `(i, j)` of the results grid
            horizontal_start = i * size + i * margin
            horizontal_end = horizontal_start + size
            vertical_start = j * size + j * margin
            vertical_end = vertical_start + size
            
            results[horizontal_start: horizontal_end, vertical_start: vertical_end, :] = filter_img
            
            
    # Display the results grid
    plt.figure(figsize=(20, 20))
    plt.imshow(results)
    plt.show()

These filter visualizations tell you a lot about how convnet layers see the world: each layer in a convnet learns a collection of filters such that their inputs can be expressed as a combination of the filters. This is similar to how the Fourier transform decomposes signals onto a bank of cosine functions. The filters in these convnet filter banks get increasingly complex and refined as you go higher in the model:

* The filters from the first layer in the model (block1_conv1) encode simple directional edges and colors (or colored edges, in some cases).
* The filters from block2_conv1 encode simple textures made from combinations of edges and colors. 
* The filters in higher layers begin to resemble textures found in natural images: feathers, eyes, leaves, and so on.

**Visualizing heatmaps of class activation **

I’ll introduce one more visualization technique: one that is useful for understanding
which parts of a given image led a convnet to its final classification decision. This is helpful for debugging the decision process of a convnet, particularly in the case of a classification mistake. It also allows you to locate specific objects in an image. This general category of techniques is called class activation map (CAM) visualization, and it consists of producing heatmaps of class activation over input images. A class activation heatmap is a 2D grid of scores associated with a specific output class, computed for every location in any input image, indicating how important each location is with respect to the class under consideration.


The specific implementation you’ll use is the one described in “Grad-CAM: Visual Explanations from Deep Networks via Gradient-based Localization.”2 It’s very simple: it consists of taking the output feature map of a convolution layer, given an input image, and weighing every channel in that feature map by the gradient of the class with respect to the channel. Intuitively, one way to understand this trick is that you’re weighting a spatial map of “how intensely the input image activates different channels” by “how important each channel is with regard to the class,” resulting in a spatial map of “how intensely the input image activates the class.”

In [ ]:
from keras.applications.vgg16 import VGG16 # importing VGG16
model = VGG16(weights='imagenet') # loading the model with the weighhts used in Imagenet

Now we need to convert a image into numpy array  of size 224 × 244, convert it to a Numpy float32 tensor, and apply
these preprocessing rules.


In [ ]:
# importing preprocess_input, decode_predictions from vgg16
from keras.applications.vgg16 import preprocess_input, decode_predictions

# setting the image path and converting teh image to an array

img_path = '../input/african-elephant/antelope-park-6Rbqy9jS7xs-unsplash.jpg'
img = image.load_img(img_path, target_size=(224, 224)) # reshaped into size of 224 x 224
x = image.img_to_array(img) # converting the values into tensors
x = np.expand_dims(x, axis=0) # expanding it to the required dimension
x = preprocess_input(x)




You can now run the pretrained network on the image and decode its prediction vector back to a human-readable format:

In [ ]:
preds = model.predict(x) # predicting based on the model

In [ ]:
 print('Predicted:', decode_predictions(preds, top=3)[0]) # checking the preictions

The top three classes predicted for this image are as follows:

* African elephant (with 92.5% probability)
* Tusker (with 7% probability)
* Indian elephant (with 0.4% probability)

The network has recognized the image as containing an undetermined quantity of African elephants. The entry in the prediction vector that was maximally activated is the one corresponding to the “African elephant” class.

In [ ]:
 np.argmax(preds[0]) 

To visualize which parts of the image are the most African elephant–like, let’s set up the Grad-CAM process.

In [ ]:
# using the grad- cam process to visualize partsof the image
african_elephant_output = model.output[:, 386]
last_conv_layer = model.get_layer('block5_conv3')
grads = K.gradients(african_elephant_output, last_conv_layer.output)[0]
pooled_grads = K.mean(grads, axis=(0, 1, 2))
iterate = K.function([model.input],
                     [pooled_grads, last_conv_layer.output[0]])
pooled_grads_value, conv_layer_output_value = iterate([x])
for i in range(512):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

# making a heatmap of the output from the network
heatmap = np.mean(conv_layer_output_value, axis=-1)

For visualization purposes, you’ll also normalize the heatmap between 0 and 1. 

In [ ]:
# normalizing the heatmap
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
plt.matshow(heatmap)

Finally, you’ll use OpenCV to generate an image that superimposes the original image on the heatmap you just obtained.

In [ ]:
# importing opencv to look into the superimposed image
import cv2
img = cv2.imread(img_path)
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
superimposed_img = heatmap * 0.4 + img

cv2.imwrite('./elephant_cam.jpg', superimposed_img)

So this how the ConvNet identified that it is an African Elephant. Superimposition is really a great tool to visualize the learning as to why I guess it is pretty visible if you just download and check out the heatmap superimposed elephant image